# Modelltraining


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV # NEU: TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
from sklearn.inspection import permutation_importance

# --- Konfiguration ---
PREPARED_DATA_FOLDER = "financial_data_prepared"
TICKER_SYMBOL = "GOOG"
ASSET_TYPE = "stocks"

# --- Daten laden ---
prepared_filename = f"prepared_{TICKER_SYMBOL}_{ASSET_TYPE}_data.csv"
prepared_filepath = os.path.join(PREPARED_DATA_FOLDER, prepared_filename)

print(f"Lade vorbereitete Daten von: {prepared_filepath}")
try:
    df = pd.read_csv(
        prepared_filepath,
        index_col='Date',
        parse_dates=True
    )
    print(f"Datei erfolgreich geladen. Spalten: {df.columns.tolist()}")
except FileNotFoundError:
    print(f"FEHLER: Datei nicht gefunden unter: {prepared_filepath}")
    print("Stelle sicher, dass das Haupt-Datenaufbereitungsskript erfolgreich durchlief")
    print(f"und die Datei im Ordner '{os.path.dirname(prepared_file_path)}' erstellt wurde.")
    exit()
except Exception as e:
    print(f"Ein anderer Fehler trat beim Laden der Datei auf: {e}")
    exit()


# --- Preisspalte bestimmen ---
if 'Adj Close' in df.columns:
    price_col = 'Adj Close'
elif 'Close' in df.columns:
    price_col = 'Close'
else:
    print("FEHLER: Weder 'Adj Close' noch 'Close' Spalte gefunden!")
    exit()
print(f"Verwende '{price_col}' als Preisspalte.")


# --- Zusätzliche Features erstellen ---
if 'Upper' in df.columns and 'Lower' in df.columns:
    df['Close_minus_Upper'] = df[price_col] - df['Upper']
    df['Close_minus_Lower'] = df[price_col] - df['Lower']
    df['Band_Width'] = df['Upper'] - df['Lower']
    df['Position_in_Band'] = ((df[price_col] - df['Lower']) / df['Band_Width']).replace([np.inf, -np.inf], np.nan)
else:
    print("Warnung: 'Upper' oder 'Lower' Spalte nicht in CSV gefunden. Bollinger-Features können nicht erstellt werden.")

df['Momentum'] = df[price_col] - df[price_col].rolling(window=12).mean()


# --- Zielvariable definieren ---
df['Next_Day_Price'] = df[price_col].shift(-1)
df['Target'] = (df['Next_Day_Price'] > df[price_col]).astype(int)


# --- Datenbereinigung ---
print(f"Zeilen vor dropna: {len(df)}")
df.dropna(inplace=True)
print(f"Zeilen nach dropna: {len(df)}")


if df.empty:
    print("FEHLER: Keine Daten mehr nach Bereinigung!")
    exit()

# --- Features und Zielvariable definieren ---
features = []
potential_features = [
    'Close_minus_Upper', 'Close_minus_Lower', 'Band_Width',
    'Position_in_Band',
    'SMA_50',
    'SMA_200',
    'Momentum',
    # --- HIER HINZUFÜGEN ---
    'RSI',           # RSI hinzufügen
    'MACD',          # MACD-Linie hinzufügen
    'Signal',        # MACD-Signallinie hinzufügen (oft auch nützlich)
    'MACD_Histogram' # MACD-Histogramm hinzufügen (oft auch nützlich)
    # ----------------------
]

# Nur Features hinzufügen, die im DataFrame existieren 
for feature in potential_features:
    if feature in df.columns:
        features.append(feature)
    else:
        print(f"WARNUNG: Benötigtes Feature '{feature}' nicht im DataFrame gefunden und übersprungen!")

missing_required = [f for f in potential_features if f not in df.columns]
if missing_required:
     print(f"FEHLER: Die folgenden benötigten Features fehlen im DataFrame: {missing_required}")
     # exit()

if not features:
    print("FEHLER: Keine gültigen Features für das Training gefunden!")
    exit()

print(f"Verwendete Features für das Modell (gemäß deiner Auswahl): {features}")

X = df[features]
y = df['Target']

# --- Daten aufteilen ---
# Wichtig: Split VOR der Hyperparameter-Suche, damit auf dem Test-Set final validiert wird
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"Trainingsdaten Größe: {len(X_train)}, Testdaten Größe: {len(X_test)}")

if len(X_train) == 0 or len(X_test) == 0:
    print("FEHLER: Trainings- oder Testdatensatz ist nach dem Splitten leer.")
    exit()


# --- NEU: Hyperparameter-Optimierung ---
print("\nStarte Hyperparameter-Optimierung (GridSearchCV)...")

# 1. Parameter-Gitter definieren (Beispiel - anpassen nach Bedarf)
#    Reduziertes Grid für schnellere Demonstration
param_grid = {
    'n_estimators': [100, 200],             # Anzahl Bäume
    'max_depth': [None, 10, 20],            # Max Tiefe pro Baum (None = unbegrenzt)
    'min_samples_split': [2, 5],            # Min Samples für Split
    'min_samples_leaf': [1, 3],             # Min Samples pro Blatt
    'max_features': ['sqrt', None]          # Anzahl Features pro Split ('sqrt' ist oft gut)
    # 'class_weight': [None, 'balanced']    # Optional: Bei unbalancierten Klassen testen
}

# 2. Zeitreihen-Kreuzvalidierungs-Splitter erstellen
#    Splits die Trainingsdaten in mehrere aufeinanderfolgende Train/Validation-Folds
tscv = TimeSeriesSplit(n_splits=5) # z.B. 5 Folds

# 3. GridSearchCV-Objekt erstellen
#    (Alternative: RandomizedSearchCV für schnellere Suche bei großen Grids)
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), # Das zu optimierende Modell
    param_grid=param_grid,                             # Das Suchgitter
    cv=tscv,                                           # Die Kreuzvalidierungs-Strategie
    scoring='accuracy',                                # Die zu optimierende Metrik
    n_jobs=-1,                                         # Nutze alle CPU-Kerne
    verbose=1                                          # Zeige Fortschritt an
)

# 4. Suche auf den Trainingsdaten durchführen
#    Findet die besten Parameter basierend auf der durchschnittlichen Performance
#    über die Kreuzvalidierungs-Folds HINNERHALB der Trainingsdaten.
grid_search.fit(X_train, y_train)

# 5. Beste gefundene Parameter und bestes Modell extrahieren
print(f"\nBeste gefundene Parameter durch GridSearchCV: {grid_search.best_params_}")
print(f"Beste durchschnittliche Cross-Validation Accuracy: {grid_search.best_score_:.4f}")
best_model = grid_search.best_estimator_ # Das ist das RandomForest-Modell mit den besten Parametern

# --- Modell trainieren ---
# Dieser Schritt ist jetzt implizit durch grid_search.fit() passiert.
# best_model ist bereits auf den (besten Teilen der) Trainingsdaten trainiert.
# Man könnte optional best_model.fit(X_train, y_train) erneut aufrufen, um sicherzustellen,
# dass es auf den *gesamten* Trainingsdaten mit den besten Parametern trainiert ist.
# Das ist oft gängige Praxis nach einer CV-Suche.
print("\nTrainiere finales Modell mit besten Parametern auf gesamten Trainingsdaten...")
best_model.fit(X_train, y_train)
print("Finales Modell trainiert.")


# --- Vorhersagen machen (mit dem optimierten Modell) ---
y_pred = best_model.predict(X_test)

# --- Bewertung (des optimierten Modells) ---
accuracy = accuracy_score(y_test, y_pred)
print(f'\nGenauigkeit des OPTIMIERTEN Modells auf den Testdaten: {accuracy:.4f}')

# --- Feature Importance (vom optimierten Modell) ---
if hasattr(best_model, 'feature_importances_'):
    print("\nFeature Importance (Mean Decrease Impurity - Optimiertes Modell):")
    if len(features) == best_model.n_features_in_:
         mdi_importances = pd.Series(best_model.feature_importances_, index=features).sort_values(ascending=False)
         print(mdi_importances)
    else:
        print("Warnung: Anzahl der Features stimmt nicht mit Modell überein, MDI Importance wird nicht angezeigt.")

# --- Permutation Importance (vom optimierten Modell) ---
print("\nBerechne Permutation Importance für optimiertes Modell (kann einen Moment dauern)...")
perm_importance_result = permutation_importance(
    estimator=best_model, # BENUTZE DAS OPTIMIERTE MODELL
    X=X_test,
    y=y_test,
    scoring='accuracy',
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

# Ergebnisse aufbereiten und anzeigen
perm_sorted_idx = perm_importance_result.importances_mean.argsort()[::-1]

print("\nPermutation Importance (Optimiertes Modell, basierend auf Testset-Accuracy):")
for i in perm_sorted_idx:
    feature_name = features[i]
    mean_importance = perm_importance_result.importances_mean[i]
    std_importance = perm_importance_result.importances_std[i]
    print(f"  {feature_name:<20}: {mean_importance:.4f} +/- {std_importance:.4f}")


print("\n=== Skriptausführung beendet ===")

Lade vorbereitete Daten von: financial_data_prepared\prepared_GOOG_stocks_data.csv
Datei erfolgreich geladen. Spalten: ['Close', 'High', 'Low', 'Open', 'SMA_20', 'SMA_50', 'SMA_200', 'StdDev_20', 'Upper', 'Lower', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal', 'MACD_Histogram']
Verwende 'Close' als Preisspalte.
Zeilen vor dropna: 1147
Zeilen nach dropna: 1135
Verwendete Features für das Modell (gemäß deiner Auswahl): ['Close_minus_Upper', 'Close_minus_Lower', 'Band_Width', 'Position_in_Band', 'SMA_50', 'SMA_200', 'Momentum', 'RSI', 'MACD', 'Signal', 'MACD_Histogram']
Trainingsdaten Größe: 908, Testdaten Größe: 227

Starte Hyperparameter-Optimierung (GridSearchCV)...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Beste gefundene Parameter durch GridSearchCV: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Beste durchschnittliche Cross-Validation Accuracy: 0.5113

Trainiere finales Modell mit besten Parametern 